In [ ]:
import torch

args = {
    'model_path': 'meta-llama/Meta-Llama-3.1-8B-Instruct',
    'task': 'text-generation',
    'torch_dtype': torch.bfloat16,
    'device_map': 'auto',
    'do_sample': True,
    'temperature': 0.1,
    'max_new_tokens': 700,
    'num_gpus': 1,
    'load_8bit': False,
    'cpu_offloading': False,
    'max_gpu_memory': None,
    'revision': 'main',
    'debug': False,
    'top_p': 1,
    'full_texts_dir': '/content/drive/MyDrive/AI/MultiClinSum 2025/Data/fulltext',
    'output_dir': '/content/drive/MyDrive/AI/MultiClinSum 2025/Results/Baseline'
}

class DotDict:
    def __init__(self, dictionary):
        self._data = dictionary

    def __getattr__(self, key):
        if key in self._data:
            return self._data[key]
        else:
            raise AttributeError(f"'DotDict' object has no attribute '{key}'")

    def __setattr__(self, key, value):
        if key == '_data':
            super().__setattr__(key, value)
        else:
            self._data[key] = value

    def __delattr__(self, key):
        if key in self._data:
            del self._data[key]
        else:
            raise AttributeError(f"'DotDict' object has no attribute '{key}'")

args = DotDict(args)

In [ ]:
from transformers import pipeline

summarization_pipeline = pipeline(
    args.task,
    model=args.model_path,
    model_kwargs={"torch_dtype": args.torch_dtype},
    device_map=args.device_map,
    max_new_tokens=args.max_new_tokens,
    do_sample=args.do_sample,
    temperature=args.temperature,
    top_p=args.top_p
)

In [ ]:
def get_summary(text_to_summarize):
  messages = [
  {
    "role": "system",
    "content": "Eres un asistente médico experto en redactar resúmenes clínicos concisos y precisos en español. Tu tarea es analizar informes de casos clínicos y generar un resumen narrativo que destaque los aspectos más relevantes para profesionales de la salud, incluyendo hallazgos médicos clave como resultados de pruebas importantes y otras observaciones significativas. El resumen debe presentarse en un solo párrafo, sin utilizar viñetas, encabezados ni ningún tipo de formato especial."
  },
  {
    "role": "user",
    "content": f"Por favor, resume el siguiente informe de caso clínico en español, enfocándote en:\n\n- Datos clínicos clave\n- Resultados de pruebas importantes (laboratorio, imagenología, etc.)\n- Diagnóstico\n- Tratamiento\n- Evolución del paciente\n\nAsegúrate de que el resumen sea claro, coherente y útil para profesionales médicos. Presenta el resumen en un solo párrafo, sin utilizar viñetas, encabezados ni ningún tipo de formato especial.\n\n{text_to_summarize}"
  }
 ]

  return summarization_pipeline(messages)[0]["generated_text"][-1]["content"]

In [ ]:
import os

done = list(os.listdir(args.output_dir))

full_text_file_names_sorted = sorted(
    [f for f in os.listdir(args.full_texts_dir) if f.endswith(".txt")],
    key=lambda x: int(x.split('_')[-1].removesuffix('.txt'))
)

for file_name in full_text_file_names_sorted:
  if file_name in done:
    print("Skipping", file_name)
    continue

  print("Summarizing", file_name)
  with open(f"{args.full_texts_dir}/{file_name}", "r") as in_file:
    with open(f"{args.output_dir}/{file_name}", "w") as out_file:
      out_file.write(get_summary(in_file.read()))